## Authorization to the DEDL services

In [1]:
from dedllogin import DEDL_auth
import requests
import json
import os
from getpass import getpass
from IPython.display import JSON

username = input("Please input your DESP username or email: ")
password = getpass("Please input your DESP password: ")

auth = DEDL_auth(username, password)

access_token = auth.get_token()
if access_token is not None:
    print("DEDL/DESP Access Token Obtained Successfully")
else:
    print("Failed to Obtain DEDL/DESP Access Token")

auth_headers = {"Authorization": f"Bearer {access_token}"}

Please input your DESP username or email:  jesper.dejby@ri.se
Please input your DESP password:  ········


DEDL/DESP Access Token Obtained Successfully


## Download data

We are aiming to download the ODYSSEA Global Ocean dataset described here: https://hda.data.destination-earth.eu/ui/dataset/EO.MO.DAT.SST_GLO_SST_L3S_NRT_OBSERVATIONS_010_010

In [6]:
response_sst = requests.post("https://hda.data.destination-earth.eu/stac/search", headers=auth_headers, json={
    "collections": ["EO.MO.DAT.SST_GLO_SST_L3S_NRT_OBSERVATIONS_010_010"],
    "datetime": "2024-05-31T00:00:00Z/2024-06-01T00:00:00Z",
    'bbox': [10,53,30,66]
})

In [7]:
from IPython.display import JSON
JSON(response_sst.json())

<IPython.core.display.JSON object>

In [8]:
response_sst.json()["features"][0]["assets"]["downloadLink"]["href"]

'https://hda.data.destination-earth.eu/stac/collections/EO.MO.DAT.SST_GLO_SST_L3S_NRT_OBSERVATIONS_010_010/items/20240531000000-IFR-L3S_GHRSST-SSTfnd-ODYSSEA-GLOB_010-v02/download?provider=cop_marine'

In [9]:
product_sst = []
assets = []
for i in response_sst.json()["features"]:
    # Download NC
    #product_sst.append(i["assets"]["downloadLink"]["alternate"]["origin"]["href"])
    # Download using main link
    product_sst.append(i["assets"]["downloadLink"]["href"])
    assets.append(i["id"])

In [10]:
from tqdm import tqdm
import time

# Define a list of assets to download
download_url = product_sst[0]
filename = assets[0]
response = requests.get(download_url, headers=auth_headers)
total_size = int(response.headers.get("content-length", 0))

print(f"downloading {filename}")

with tqdm(total=total_size, unit="B", unit_scale=True) as progress_bar:
    with open(filename, 'wb') as f:
        for data in response.iter_content(1024):
            progress_bar.update(len(data))
            f.write(data)

downloading 20240531000000-IFR-L3S_GHRSST-SSTfnd-ODYSSEA-GLOB_010-v02


25.2MB [00:00, 482MB/s]


## Read data

We read the data and subset it too the baltic sea region.

In [11]:
import rioxarray
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import netCDF4 as nc

In [15]:
ds = xr.open_dataset("20240531000000-IFR-L3S_GHRSST-SSTfnd-ODYSSEA-GLOB_010-v02")

In [16]:
ds

<xarray.Dataset>
Dimensions:                           (lat: 1600, lon: 3600, time: 1)
Coordinates:
  * lat                               (lat) float32 -79.95 -79.85 ... 79.95
  * lon                               (lon) float32 -179.9 -179.9 ... 179.9
  * time                              (time) datetime64[ns] 2024-05-31
Data variables: (12/15)
    crs                               int64 ...
    sea_surface_temperature           (time, lat, lon) float32 ...
    or_number_of_pixels               (time, lat, lon) float32 ...
    sea_surface_temperature_stddev    (time, lat, lon) float32 ...
    quality_level                     (time, lat, lon) float32 ...
    sses_bias                         (time, lat, lon) float32 ...
    ...                                ...
    or_longitude                      (time, lat, lon) float32 ...
    adjusted_sea_surface_temperature  (time, lat, lon) float32 ...
    solar_zenith_angle                (time, lat, lon) float32 ...
    bias_to_reference_sst             (time, lat, lon) float32 ...
    sources_of_sst                    (time, lat, lon) float32 ...
    sst_dtime                         (time, lat, lon) float64 ...
Attributes: (12/74)
    file_quality_level:              3
    title:                           ODYSSEA Global Sea Surface Temperature G...
    id:                              ODYSSEA-IFR-L3S-GLO_010-v3
    cmems_product_id:                SST_GLO_SST_L3S_NRT_OBSERVATIONS_010_010
    reference:                       Product User Manual for L3S Odyssea Prod...
    summary:                         This dataset provide a times series of d...
    ...                              ...
    geospatial_lon_units:            degrees_east
    geospatial_vertical_min:         0.0
    geospatial_vertical_max:         0.0
    temporal_resolution:             daily
    time_coverage_resolution:        P1D
    time_coverage_duration:          P1D

In [17]:
lat_bnds, lon_bnds = [53, 66], [10, 30]
baltic_sea_subset=ds.sel(lat=slice(*lat_bnds), lon=slice(*lon_bnds))

In [18]:
baltic_sea_subset

<xarray.Dataset>
Dimensions:                           (lat: 130, lon: 200, time: 1)
Coordinates:
  * lat                               (lat) float32 53.05 53.15 ... 65.85 65.95
  * lon                               (lon) float32 10.05 10.15 ... 29.85 29.95
  * time                              (time) datetime64[ns] 2024-05-31
Data variables: (12/15)
    crs                               int64 ...
    sea_surface_temperature           (time, lat, lon) float32 ...
    or_number_of_pixels               (time, lat, lon) float32 ...
    sea_surface_temperature_stddev    (time, lat, lon) float32 ...
    quality_level                     (time, lat, lon) float32 ...
    sses_bias                         (time, lat, lon) float32 ...
    ...                                ...
    or_longitude                      (time, lat, lon) float32 ...
    adjusted_sea_surface_temperature  (time, lat, lon) float32 ...
    solar_zenith_angle                (time, lat, lon) float32 ...
    bias_to_reference_sst             (time, lat, lon) float32 ...
    sources_of_sst                    (time, lat, lon) float32 ...
    sst_dtime                         (time, lat, lon) float64 ...
Attributes: (12/74)
    file_quality_level:              3
    title:                           ODYSSEA Global Sea Surface Temperature G...
    id:                              ODYSSEA-IFR-L3S-GLO_010-v3
    cmems_product_id:                SST_GLO_SST_L3S_NRT_OBSERVATIONS_010_010
    reference:                       Product User Manual for L3S Odyssea Prod...
    summary:                         This dataset provide a times series of d...
    ...                              ...
    geospatial_lon_units:            degrees_east
    geospatial_vertical_min:         0.0
    geospatial_vertical_max:         0.0
    temporal_resolution:             daily
    time_coverage_resolution:        P1D
    time_coverage_duration:          P1D

## Final remarks

In the end, we decided to limit our scope and not include a sea surface temperature dataset.